In [ ]:
# !pip install -q -U pillow git+https://github.com/huggingface/transformers git+https://github.com/huggingface/peft.git datasets pymupdf bitsandbytes seaborn matplotlib

## Load Data

In [1]:
from PIL import Image

img = Image.open("/Users/chenghao/Desktop/WechatIMG18.jpg")

## Load Model

In [ ]:
import torch
from transformers import AutoProcessor, Idefics2ForConditionalGeneration, BitsAndBytesConfig

base_model = "HuggingFaceM4/idefics2-8b"
# peft_model_id = "chenghao/idefics2-edgar"
peft_model_id = "HuggingFaceM4/idefics2-8b"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = Idefics2ForConditionalGeneration.from_pretrained(
    peft_model_id,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

model.eval()

## Evaluation

In [ ]:
from tqdm.auto import tqdm
import time

processor = AutoProcessor.from_pretrained(
    base_model,
    do_image_splitting=True,
    # size={"longest_edge": 490, "shortest_edge": 350}
)
images = [img]
question = "Convert the table into HTML code"
messages = [
    {
        "role": "user",
        "content": [{"type": "image"} for _ in range(len(images))] + [{"type": "text", "text": question}],
    },
]

prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=images, return_tensors="pt").to("cuda")
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=2048)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
preds = [t.split("Assistant:", 1)[-1].strip() for t in generated_texts]